In [10]:
import openpyxl
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor

from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

from matplotlib import pyplot as plt
import datetime
import math
import pandas as pd
from scipy import stats
import seaborn as sn

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [307]:
# The first egg dataset
egg_1_dataframe = pd.read_csv('egg_1_06_05_2020_25_05_2020.csv')
egg_1_dataframe = egg_1_dataframe.drop(columns=['Latitude[deg]', 'Longitude[deg]', 'Altitude[m]'])
egg_1_dataframe['Timestamp'] = pd.to_datetime(egg_1_dataframe['Timestamp'])
egg_1_dataframe = egg_1_dataframe.resample('60min', on='Timestamp').mean()
egg_1_dataframe.to_pickle('egg_1_06_05_2020_25_05_2020_hourly.pkl')

# The second egg dataset
egg_2_dataframe = pd.read_csv('egg_2_06_05_2020_25_05_2020.csv')
egg_2_dataframe = egg_2_dataframe.drop(columns=['Latitude[deg]', 'Longitude[deg]', 'Altitude[m]'])
egg_2_dataframe['Timestamp'] = pd.to_datetime(egg_2_dataframe['Timestamp'])
egg_2_dataframe = egg_2_dataframe.resample('60min', on='Timestamp').mean()
egg_2_dataframe.to_pickle('egg_2_06_05_2020_25_05_2020_hourly.pkl')

# The third egg dataset
egg_3_dataframe = pd.read_csv('egg_2_06_05_2020_25_05_2020.csv')
egg_3_dataframe = egg_3_dataframe.drop(columns=['Latitude[deg]', 'Longitude[deg]', 'Altitude[m]'])
egg_3_dataframe['Timestamp'] = pd.to_datetime(egg_3_dataframe['Timestamp'])
egg_3_dataframe = egg_3_dataframe.resample('60min', on='Timestamp').mean()
egg_3_dataframe.to_pickle('egg_3_06_05_2020_25_05_2020_hourly.pkl')

# The airpointer dataset
airpointer_dataframe = pd.read_csv('Airpointer_01_05_2020_02_06_2020.csv')
airpointer_dataframe['Time'] = pd.to_datetime(airpointer_dataframe['Time'])
airpointer_dataframe = airpointer_dataframe.resample('60min', on='Time').mean()
airpointer_dataframe.to_pickle('airpointer_01_05_2020_02_06_2020_hourly.pkl')

In [239]:
egg_1_dataframe.head()

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa]
Timestamp,,,,,,,,,,
2020-05-06 13:00:00,20.520000,20.116667,3.166667,3.666667,4.000000,142.300000,1.421641,1935.236667,1.410120,99261.900000
2020-05-06 14:00:00,17.968333,24.757292,13.868750,15.925000,16.810417,63.958958,1.423080,263.038333,1.422146,99264.718750
2020-05-06 15:00:00,22.348833,19.432667,1.673333,2.333333,2.475000,60.012667,1.423583,95.994500,1.423568,99188.643333
2020-05-06 16:00:00,22.438667,18.594000,1.186667,1.920000,2.090000,61.118000,1.423567,56.341500,1.423861,99138.068333
2020-05-06 17:00:00,22.471500,18.093833,1.471667,2.248333,2.441667,62.872167,1.423533,49.240500,1.423915,99108.190000


In [241]:
egg_2_dataframe.head()

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa]
Timestamp,,,,,,,,,,
2020-05-06 13:00:00,21.180000,23.270000,1.000000,1.500000,1.500000,132.520000,1.424859,2664.450000,1.397469,99291.800000
2020-05-06 14:00:00,18.300385,24.459423,1.203846,2.282692,2.573077,57.853654,1.426289,380.277692,1.414856,99296.513462
2020-05-06 15:00:00,21.573667,19.899667,1.035000,1.966667,2.193333,49.278667,1.426699,166.483333,1.416678,99223.611667
2020-05-06 16:00:00,20.936500,19.845833,1.188333,2.235000,2.576667,50.088667,1.426638,159.735500,1.416693,99177.978333
2020-05-06 17:00:00,20.838500,19.497333,1.536667,2.705000,3.088333,49.248667,1.426650,146.279333,1.416790,99149.185000


In [253]:
egg_3_dataframe.head()

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa]
Timestamp,,,,,,,,,,
2020-05-06 13:00:00,21.180000,23.270000,1.000000,1.500000,1.500000,132.520000,1.424859,2664.450000,1.397469,99291.800000
2020-05-06 14:00:00,18.300385,24.459423,1.203846,2.282692,2.573077,57.853654,1.426289,380.277692,1.414856,99296.513462
2020-05-06 15:00:00,21.573667,19.899667,1.035000,1.966667,2.193333,49.278667,1.426699,166.483333,1.416678,99223.611667
2020-05-06 16:00:00,20.936500,19.845833,1.188333,2.235000,2.576667,50.088667,1.426638,159.735500,1.416693,99177.978333
2020-05-06 17:00:00,20.838500,19.497333,1.536667,2.705000,3.088333,49.248667,1.426650,146.279333,1.416790,99149.185000


In [277]:
df3 = pd.concat((egg_1_dataframe, egg_2_dataframe, egg_3_dataframe))
egg_combined = df3.groupby(df3.index).mean()
egg_combined

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa]
Timestamp,,,,,,,,,,
2020-05-06 13:00:00,20.960000,22.218889,1.722222,2.222222,2.333333,135.780000,1.423786,2421.378889,1.401686,99281.833333
2020-05-06 14:00:00,18.189701,24.558713,5.425481,6.830128,7.318857,59.888755,1.425219,341.197906,1.417286,99285.915224
2020-05-06 15:00:00,21.832056,19.744000,1.247778,2.088889,2.287222,52.856667,1.425660,142.987056,1.418974,99211.955556
2020-05-06 16:00:00,21.437222,19.428556,1.187778,2.130000,2.414444,53.765111,1.425614,125.270833,1.419082,99164.675000
2020-05-06 17:00:00,21.382833,19.029500,1.515000,2.552778,2.872778,53.789833,1.425611,113.933056,1.419165,99135.520000
...,...,...,...,...,...,...,...,...,...,...
2020-05-25 11:00:00,39.957278,30.284889,5.142222,6.795000,7.046111,69.635444,1.427285,107.110611,1.420837,99719.558889
2020-05-25 12:00:00,39.840944,30.209333,6.118333,7.976667,8.320000,75.164167,1.427154,124.671500,1.420680,99715.847778
2020-05-25 13:00:00,36.916722,36.016833,11.229444,15.010556,15.527222,78.477944,1.426714,118.204833,1.420419,99704.912222


In [278]:
egg_combined.to_pickle('master_dataset.pkl')

In [280]:
egg_combined = pd.read_pickle('master_dataset.pkl')
egg_combined

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa]
Timestamp,,,,,,,,,,
2020-05-06 13:00:00,20.960000,22.218889,1.722222,2.222222,2.333333,135.780000,1.423786,2421.378889,1.401686,99281.833333
2020-05-06 14:00:00,18.189701,24.558713,5.425481,6.830128,7.318857,59.888755,1.425219,341.197906,1.417286,99285.915224
2020-05-06 15:00:00,21.832056,19.744000,1.247778,2.088889,2.287222,52.856667,1.425660,142.987056,1.418974,99211.955556
2020-05-06 16:00:00,21.437222,19.428556,1.187778,2.130000,2.414444,53.765111,1.425614,125.270833,1.419082,99164.675000
2020-05-06 17:00:00,21.382833,19.029500,1.515000,2.552778,2.872778,53.789833,1.425611,113.933056,1.419165,99135.520000
...,...,...,...,...,...,...,...,...,...,...
2020-05-25 11:00:00,39.957278,30.284889,5.142222,6.795000,7.046111,69.635444,1.427285,107.110611,1.420837,99719.558889
2020-05-25 12:00:00,39.840944,30.209333,6.118333,7.976667,8.320000,75.164167,1.427154,124.671500,1.420680,99715.847778
2020-05-25 13:00:00,36.916722,36.016833,11.229444,15.010556,15.527222,78.477944,1.426714,118.204833,1.420419,99704.912222


In [281]:
egg_combined['Airpointer_NO2[ppb]'] = 0

In [282]:
egg_combined

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa],Airpointer_NO2[ppb]
Timestamp,,,,,,,,,,,
2020-05-06 13:00:00,20.960000,22.218889,1.722222,2.222222,2.333333,135.780000,1.423786,2421.378889,1.401686,99281.833333,0
2020-05-06 14:00:00,18.189701,24.558713,5.425481,6.830128,7.318857,59.888755,1.425219,341.197906,1.417286,99285.915224,0
2020-05-06 15:00:00,21.832056,19.744000,1.247778,2.088889,2.287222,52.856667,1.425660,142.987056,1.418974,99211.955556,0
2020-05-06 16:00:00,21.437222,19.428556,1.187778,2.130000,2.414444,53.765111,1.425614,125.270833,1.419082,99164.675000,0
2020-05-06 17:00:00,21.382833,19.029500,1.515000,2.552778,2.872778,53.789833,1.425611,113.933056,1.419165,99135.520000,0
...,...,...,...,...,...,...,...,...,...,...,...
2020-05-25 11:00:00,39.957278,30.284889,5.142222,6.795000,7.046111,69.635444,1.427285,107.110611,1.420837,99719.558889,0
2020-05-25 12:00:00,39.840944,30.209333,6.118333,7.976667,8.320000,75.164167,1.427154,124.671500,1.420680,99715.847778,0
2020-05-25 13:00:00,36.916722,36.016833,11.229444,15.010556,15.527222,78.477944,1.426714,118.204833,1.420419,99704.912222,0


In [283]:
egg_combined.to_pickle('master_dataset.pkl')

In [291]:
egg_combined = pd.read_pickle('master_dataset.pkl')
egg_combined.tail()

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa],Airpointer_NO2[ppb]
Timestamp,,,,,,,,,,,
2020-05-25 11:00:00,39.957278,30.284889,5.142222,6.795000,7.046111,69.635444,1.427285,107.110611,1.420837,99719.558889,0
2020-05-25 12:00:00,39.840944,30.209333,6.118333,7.976667,8.320000,75.164167,1.427154,124.671500,1.420680,99715.847778,0
2020-05-25 13:00:00,36.916722,36.016833,11.229444,15.010556,15.527222,78.477944,1.426714,118.204833,1.420419,99704.912222,0
2020-05-25 14:00:00,35.323556,38.016611,11.071111,15.091667,15.686111,76.535778,1.426550,95.154722,1.420436,99690.397222,0
2020-05-25 15:00:00,36.740904,35.649444,9.213778,12.565333,13.173852,76.635585,1.426728,79.540526,1.420712,99679.745037,0


In [287]:
airpointer_dataframe.index

DatetimeIndex(['2020-05-01 10:00:00', '2020-05-01 11:00:00',
               '2020-05-01 12:00:00', '2020-05-01 13:00:00',
               '2020-05-01 14:00:00', '2020-05-01 15:00:00',
               '2020-05-01 16:00:00', '2020-05-01 17:00:00',
               '2020-05-01 18:00:00', '2020-05-01 19:00:00',
               ...
               '2020-06-02 05:00:00', '2020-06-02 06:00:00',
               '2020-06-02 07:00:00', '2020-06-02 08:00:00',
               '2020-06-02 09:00:00', '2020-06-02 10:00:00',
               '2020-06-02 11:00:00', '2020-06-02 12:00:00',
               '2020-06-02 13:00:00', '2020-06-02 14:00:00'],
              dtype='datetime64[ns]', name='Time', length=773, freq='60T')

In [289]:
airpointer_dataframe = airpointer_dataframe.drop(airpointer_dataframe[airpointer_dataframe.index < '2020-05-06 13:00:00'].index)

In [309]:
airpointer_dataframe = airpointer_dataframe.drop(airpointer_dataframe[airpointer_dataframe.index < '2020-05-06 13:00:00'].index)

In [313]:
# The airpointer dataset
airpointer_dataframe = pd.read_csv('Airpointer_01_05_2020_02_06_2020.csv')
airpointer_dataframe['Time'] = pd.to_datetime(airpointer_dataframe['Time'])
airpointer_dataframe = airpointer_dataframe.resample('60min', on='Time').mean()
airpointer_dataframe.to_pickle('airpointer_01_05_2020_02_06_2020_hourly.pkl')

In [315]:
airpointer_dataframe

,NO,NO2,NOx,O3,PM10_conc,PM10-2.5_conc,PM2.5_conc,Air_Temperature,Relative_Humidity
Time,,,,,,,,,
2020-05-01 10:00:00,0.874167,2.238000,3.113667,21.013833,9.099500,5.401167,3.697333,9.912500,76.664333
2020-05-01 11:00:00,0.889667,2.382667,3.272167,22.627167,8.756833,4.954500,3.803833,10.319333,73.254333
2020-05-01 12:00:00,0.926667,2.418000,3.345000,27.014667,7.063833,4.724833,2.339333,11.272000,67.689667
2020-05-01 13:00:00,0.639500,1.479833,2.091500,34.241500,7.312833,5.939000,1.371333,14.311667,56.445500
2020-05-01 14:00:00,0.463167,1.222500,1.686667,38.552667,4.960500,3.578500,1.383167,15.864500,48.331167
...,...,...,...,...,...,...,...,...,...
2020-06-02 10:00:00,2.176050,7.480200,9.656100,44.714200,19.079400,13.009233,6.067050,991.048683,16.280417
2020-06-02 11:00:00,2.018567,6.709750,8.636333,48.688017,20.483817,13.730200,6.751233,990.149050,18.828600
2020-06-02 12:00:00,1.875783,6.975917,8.851783,53.534283,22.523950,15.135383,7.388267,988.823200,20.057917


In [319]:
airpointer_dataframe = pd.read_csv('Airpointer_01_05_2020_02_06_2020.csv')
airpointer_dataframe['Time'] = pd.to_datetime(airpointer_dataframe['Time'])
airpointer_dataframe = airpointer_dataframe.resample('60min', on='Time').mean()
airpointer_dataframe.loc['2020-05-06 15:00:00']

NO                    1.483333
NO2                   3.901500
NOx                   5.595000
O3                   40.472333
PM10_conc            15.562333
PM10-2.5_conc        11.270000
PM2.5_conc            4.290667
Air_Temperature      12.369500
Relative_Humidity    27.624167
Name: 2020-05-06 15:00:00, dtype: float64

In [295]:
egg_combined

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa],Airpointer_NO2[ppb]
Timestamp,,,,,,,,,,,
2020-05-06 13:00:00,20.960000,22.218889,1.722222,2.222222,2.333333,135.780000,1.423786,2421.378889,1.401686,99281.833333,0
2020-05-06 14:00:00,18.189701,24.558713,5.425481,6.830128,7.318857,59.888755,1.425219,341.197906,1.417286,99285.915224,0
2020-05-06 15:00:00,21.832056,19.744000,1.247778,2.088889,2.287222,52.856667,1.425660,142.987056,1.418974,99211.955556,0
2020-05-06 16:00:00,21.437222,19.428556,1.187778,2.130000,2.414444,53.765111,1.425614,125.270833,1.419082,99164.675000,0
2020-05-06 17:00:00,21.382833,19.029500,1.515000,2.552778,2.872778,53.789833,1.425611,113.933056,1.419165,99135.520000,0
...,...,...,...,...,...,...,...,...,...,...,...
2020-05-25 11:00:00,39.957278,30.284889,5.142222,6.795000,7.046111,69.635444,1.427285,107.110611,1.420837,99719.558889,0
2020-05-25 12:00:00,39.840944,30.209333,6.118333,7.976667,8.320000,75.164167,1.427154,124.671500,1.420680,99715.847778,0
2020-05-25 13:00:00,36.916722,36.016833,11.229444,15.010556,15.527222,78.477944,1.426714,118.204833,1.420419,99704.912222,0


In [296]:
egg_combined['Airpointer_NO2[ppb]'] = airpointer_dataframe['NO2']

In [297]:
egg_combined

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa],Airpointer_NO2[ppb]
Timestamp,,,,,,,,,,,
2020-05-06 13:00:00,20.960000,22.218889,1.722222,2.222222,2.333333,135.780000,1.423786,2421.378889,1.401686,99281.833333,3.092333
2020-05-06 14:00:00,18.189701,24.558713,5.425481,6.830128,7.318857,59.888755,1.425219,341.197906,1.417286,99285.915224,2.711000
2020-05-06 15:00:00,21.832056,19.744000,1.247778,2.088889,2.287222,52.856667,1.425660,142.987056,1.418974,99211.955556,3.901500
2020-05-06 16:00:00,21.437222,19.428556,1.187778,2.130000,2.414444,53.765111,1.425614,125.270833,1.419082,99164.675000,5.137333
2020-05-06 17:00:00,21.382833,19.029500,1.515000,2.552778,2.872778,53.789833,1.425611,113.933056,1.419165,99135.520000,7.393667
...,...,...,...,...,...,...,...,...,...,...,...
2020-05-25 11:00:00,39.957278,30.284889,5.142222,6.795000,7.046111,69.635444,1.427285,107.110611,1.420837,99719.558889,2.609533
2020-05-25 12:00:00,39.840944,30.209333,6.118333,7.976667,8.320000,75.164167,1.427154,124.671500,1.420680,99715.847778,6.032083
2020-05-25 13:00:00,36.916722,36.016833,11.229444,15.010556,15.527222,78.477944,1.426714,118.204833,1.420419,99704.912222,5.811367


In [298]:
egg_combined.to_pickle('master_dataset.pkl')

In [300]:
egg_combined = pd.read_pickle('master_dataset.pkl')
egg_combined

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa],Airpointer_NO2[ppb]
Timestamp,,,,,,,,,,,
2020-05-06 13:00:00,20.960000,22.218889,1.722222,2.222222,2.333333,135.780000,1.423786,2421.378889,1.401686,99281.833333,3.092333
2020-05-06 14:00:00,18.189701,24.558713,5.425481,6.830128,7.318857,59.888755,1.425219,341.197906,1.417286,99285.915224,2.711000
2020-05-06 15:00:00,21.832056,19.744000,1.247778,2.088889,2.287222,52.856667,1.425660,142.987056,1.418974,99211.955556,3.901500
2020-05-06 16:00:00,21.437222,19.428556,1.187778,2.130000,2.414444,53.765111,1.425614,125.270833,1.419082,99164.675000,5.137333
2020-05-06 17:00:00,21.382833,19.029500,1.515000,2.552778,2.872778,53.789833,1.425611,113.933056,1.419165,99135.520000,7.393667
...,...,...,...,...,...,...,...,...,...,...,...
2020-05-25 11:00:00,39.957278,30.284889,5.142222,6.795000,7.046111,69.635444,1.427285,107.110611,1.420837,99719.558889,2.609533
2020-05-25 12:00:00,39.840944,30.209333,6.118333,7.976667,8.320000,75.164167,1.427154,124.671500,1.420680,99715.847778,6.032083
2020-05-25 13:00:00,36.916722,36.016833,11.229444,15.010556,15.527222,78.477944,1.426714,118.204833,1.420419,99704.912222,5.811367


In [302]:
airpointer_dataframe

,NO,NO2,NOx,O3,PM10_conc,PM10-2.5_conc,PM2.5_conc,Air_Temperature,Relative_Humidity
Time,,,,,,,,,
2020-05-01 10:00:00,0.874167,2.238000,3.113667,21.013833,9.099500,5.401167,3.697333,9.912500,76.664333
2020-05-01 11:00:00,0.889667,2.382667,3.272167,22.627167,8.756833,4.954500,3.803833,10.319333,73.254333
2020-05-01 12:00:00,0.926667,2.418000,3.345000,27.014667,7.063833,4.724833,2.339333,11.272000,67.689667
2020-05-01 13:00:00,0.639500,1.479833,2.091500,34.241500,7.312833,5.939000,1.371333,14.311667,56.445500
2020-05-01 14:00:00,0.463167,1.222500,1.686667,38.552667,4.960500,3.578500,1.383167,15.864500,48.331167
...,...,...,...,...,...,...,...,...,...
2020-06-02 10:00:00,2.176050,7.480200,9.656100,44.714200,19.079400,13.009233,6.067050,991.048683,16.280417
2020-06-02 11:00:00,2.018567,6.709750,8.636333,48.688017,20.483817,13.730200,6.751233,990.149050,18.828600
2020-06-02 12:00:00,1.875783,6.975917,8.851783,53.534283,22.523950,15.135383,7.388267,988.823200,20.057917


In [303]:
airpointer_dataframe = airpointer_dataframe.drop(airpointer_dataframe[airpointer_dataframe.index < '2020-05-06 13:00:00'].index)

In [306]:
airpointer_dataframe

,NO,NO2,NOx,O3,PM10_conc,PM10-2.5_conc,PM2.5_conc,Air_Temperature,Relative_Humidity
Time,,,,,,,,,
2020-05-01 10:00:00,0.874167,2.238000,3.113667,21.013833,9.099500,5.401167,3.697333,9.912500,76.664333
2020-05-01 11:00:00,0.889667,2.382667,3.272167,22.627167,8.756833,4.954500,3.803833,10.319333,73.254333
2020-05-01 12:00:00,0.926667,2.418000,3.345000,27.014667,7.063833,4.724833,2.339333,11.272000,67.689667
2020-05-01 13:00:00,0.639500,1.479833,2.091500,34.241500,7.312833,5.939000,1.371333,14.311667,56.445500
2020-05-01 14:00:00,0.463167,1.222500,1.686667,38.552667,4.960500,3.578500,1.383167,15.864500,48.331167
...,...,...,...,...,...,...,...,...,...
2020-06-02 10:00:00,2.176050,7.480200,9.656100,44.714200,19.079400,13.009233,6.067050,991.048683,16.280417
2020-06-02 11:00:00,2.018567,6.709750,8.636333,48.688017,20.483817,13.730200,6.751233,990.149050,18.828600
2020-06-02 12:00:00,1.875783,6.975917,8.851783,53.534283,22.523950,15.135383,7.388267,988.823200,20.057917


In [320]:
# airpointer_dataframe = airpointer_dataframe.drop(airpointer_dataframe[airpointer_dataframe.index < '2020-05-06 13:00:00'].index)
# airpointer_dataframe = airpointer_dataframe.drop(airpointer_dataframe[airpointer_dataframe.index > '2020-05-25 15:00:00'].index)
# airpointer_dataframe.to_pickle('airpointer_01_05_2020_02_06_2020_hourly.pkl')

In [321]:
# Calculate the total elpased time in hours of the test
# start_time = datetime.datetime.strptime(DATA[0][0], "%Y-%m-%d %H:%M")
# elapsed_time_hours = []
# for item in DATA[:,0]:
#     hours = (datetime.datetime.strptime(item, "%Y-%m-%d %H:%M") - start_time).days * 24 + \
#     (datetime.datetime.strptime(item, "%Y-%m-%d %H:%M") - start_time).seconds / 3600   
#     elapsed_time_hours.append(hours)
# # Convert elapsed_time_hours to numpy array 
# hours = np.asarray(elapsed_time_hours, dtype='float')